# convert station id coordinates from NAD83 to WGS84

In [8]:
import ogr, osr
import gmaps
import gmaps.datasets
import os
# Find codes in this file ~/anaconda3/envs/tf2/share/gdal/pcs.csv
# 
# Using google
# WGS 84 is defined under EPSG code 4326.
# WGS84 EPSG 32618  http://www.spatialreference.org/ref/epsg/wgs-84-utm-zone-18n/

# NAD83 - EPSG:4269 - EPSG.io  possibly wrong
# NAD83  32147 http://spatialreference.org/ref/epsg/32147/

##################3
# Using the pcs.csv file

# source geogcrs code
# 4269    NAD83 lots of these
# 4326    WGS 84 lots of these


In [2]:
# Spatial Reference System
#
# fail
#inputEPSG = 32147 # NAD83
#outputEPSG = 32618 # WGS84
# fail but close
#inputEPSG = 4269 # NAD83
#outputEPSG = 4326 # WGS84
# fail
inputEPSG = 4269 # NAD83
outputEPSG = 4326 # WGS84
#

In [3]:
# Site Number: 0204300267
# Site Name: BEGGARS BRIDGE CREEK NEAR DAWLEY CORNERS, VA
# Site Type: Stream
# Agency: USGS
# Latitude 36°40'47.3",   Longitude 75°59'02.4"   NAD83
# Virginia Beach City, Virginia, Hydrologic Unit 03010205
# Datum of gage: 0.00 feet above   NAVD88.

degrees = 36
minutes = 40.0
seconds = 47.3

# According to wikipedia I should 
# use floor division for minutes
pointX_floor = degrees + minutes//60.0 + seconds/3600.0 

# According to Tuthill, don't do this
pointX = degrees + minutes/60.0 + seconds/3600.0 


degrees = 75
minutes = 59.0
seconds = 02.4

# According to wikipedia I should 
# use floor division for minutes
pointY_floor = degrees + minutes//60.0 + seconds/3600.0 

# According to Tuthill, don't do this
pointY = degrees + minutes/60.0 + seconds/3600.0 

# since the number should be negative
pointY = -pointY

print(pointX, pointY)
print("using floor {}, {}".format(pointX_floor, pointY_floor))

# The non use of floor seems to work.  Also its good enough that
# the change is coordinate systems seems to not be required.

36.679805555555554 -75.984
using floor 36.01313888888889, 75.00066666666666


In [4]:
# create a geometry from coordinates
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(pointX, pointY)

# create coordinate transformation
inSpatialRef = osr.SpatialReference()
inSpatialRef.ImportFromEPSG(inputEPSG)

outSpatialRef = osr.SpatialReference()
outSpatialRef.ImportFromEPSG(outputEPSG)

coordTransform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)

# transform point
point.Transform(coordTransform)


print(point.GetX(), point.GetY())

# a correct answer will be something like
# 36.680784, -76.003623

36.679805555555554 -75.984


In [5]:
# fwiw, here is a guy using geopandas to convert data to NAD83 and he uses 4269
# https://stackoverflow.com/questions/41820969/python-geo-spatial-coordinate-format-conversion

In [6]:
# another variant.  Essentially the same.
from osgeo import osr
inp= osr.SpatialReference()
inp.ImportFromEPSG(inputEPSG)
out= osr.SpatialReference()
out.ImportFromEPSG(outputEPSG)
transformation = osr.CoordinateTransformation(inp,out)
print(transformation.TransformPoint(pointX,pointY))
#(-74.27000000011289, 40.46999999990432, 0.0)

(36.679805555555554, -75.984, 0.0)


In [7]:
gmaps.configure(api_key=os.environ["GOOGLE_MAPS_API_KEY"])

In [16]:
fig = gmaps.figure()
# print the two locations, correct without floor and incorrect with floor
locations = [(36.679805555555554, -75.984), (36.01313888888889, -75.00066666666666)]
fig = gmaps.figure(center=(36.0, -75.0), zoom_level=8)
fig = gmaps.figure(layout={
        'width': '400px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
# give ablitly to turn on satelite
fig = gmaps.figure(map_type='HYBRID')

# show stations as green dots
stations_layer = gmaps.symbol_layer(
    locations, fill_color="green", stroke_color="green", scale=2
)
fig = gmaps.figure()
fig.add_layer(stations_layer)

fig

Figure(layout=FigureLayout(height='420px'))